In [1]:
%cd /home/yokoyama/ahar
from types import SimpleNamespace
import sys
import os
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from scipy.signal import find_peaks

sys.path.append(".")
from modules.individual import IndividualActivityRecognition, IndividualDataFormat, IndividualDataHandler
from modules.utils.constants import Stages

/raid6/home/yokoyama/ahar


In [2]:
# set constants
model_type = "ganomaly"
train_data_dir = os.path.join("data", "dataset01", "train")
test_data_dir = os.path.join("data", "dataset01", "test")
data_types = ["bbox", "local"]
seq_len = 300
masking = False

In [3]:
# load results
def load_results(data_dir, model_type, data_type, masking, seq_len, data_keys):
    video_num = os.path.basename(data_dir)
    return IndividualDataHandler.load(data_dir, model_type, data_type, masking, seq_len, data_keys)

data_keys = [IndividualDataFormat.loss_r, IndividualDataFormat.loss_d]
train_results = {}
test_results = {}
for data_type in data_types:
    data_dir = os.path.join(train_data_dir, "10")
    train_results[data_type] = load_results(data_dir, model_type, data_type, masking, seq_len, data_keys)
    data_dir = os.path.join(test_data_dir, "03")
    test_results[data_type] = load_results(data_dir, model_type, data_type, masking, seq_len, data_keys)

In [4]:
def get_results_ind(results):
    ind_results = {}
    for item in results:
        pid = item[IndividualDataFormat.id]
        if pid not in ind_results:
            # initialize individual data
            ind_results[pid] = {
                key: [] for key in IndividualDataFormat.get_keys()
                if key != IndividualDataFormat.id
            }

        # append individual data
        for key, value in item.items():
            if key != IndividualDataFormat.id:
                ind_results[pid][key].append(item[key])
    return ind_results

In [5]:
train_results_ind = {}
test_results_ind = {}
for data_type in data_types:
    train_results_ind[data_type] = get_results_ind(train_results[data_type])
    test_results_ind[data_type] = get_results_ind(test_results[data_type])

In [6]:
def calc_minmax(results):
    maxs = {data_type: {IndividualDataFormat.loss_r: 0, IndividualDataFormat.loss_d: 0} for data_type in data_types}
    mins = {data_type: {IndividualDataFormat.loss_r: 0, IndividualDataFormat.loss_d: 0} for data_type in data_types}
    for data_type, ind_results in results.items(): 
        for ind_result in ind_results.values():
            for key in [IndividualDataFormat.loss_r, IndividualDataFormat.loss_d]:
                val = ind_result[key]
                maxs[data_type][key] = max(maxs[data_type][key], max(val))
                mins[data_type][key] = min(mins[data_type][key], min(val))
    print("max", maxs)
    print("min", mins)
    
print("train")
calc_minmax(train_results_ind)
print("test")
calc_minmax(test_results_ind)

train
max {'bbox': {'loss_resitudal': 0.06646689, 'loss_discrimination': 4.7471374e-05}, 'local': {'loss_resitudal': 0.061669692, 'loss_discrimination': 3.4132972e-07}}
min {'bbox': {'loss_resitudal': 0, 'loss_discrimination': 0}, 'local': {'loss_resitudal': 0, 'loss_discrimination': 0}}
test
max {'bbox': {'loss_resitudal': 0.09098689, 'loss_discrimination': 7.309206e-05}, 'local': {'loss_resitudal': 0.20067677, 'loss_discrimination': 4.1228486e-07}}
min {'bbox': {'loss_resitudal': 0, 'loss_discrimination': 0}, 'local': {'loss_resitudal': 0, 'loss_discrimination': 0}}


# Train Data

In [25]:
peaks = find_peaks(train_results_ind["bbox"]["272"][IndividualDataFormat.loss_r], height=0.05)
peaks
# max peaks 1476, 3653

(array([1476, 1478, 1480, 1487, 1489, 1497, 1499, 3580, 3583, 3589, 3592,
        3594, 3597, 3621, 3636, 3646, 3651, 3653, 3656, 3659, 3663, 3666,
        3669]),
 {'peak_heights': array([0.0663812 , 0.06646689, 0.06633593, 0.0655917 , 0.06545711,
         0.0644491 , 0.06439633, 0.05969398, 0.0593835 , 0.05729957,
         0.05655169, 0.05640905, 0.05645631, 0.05006177, 0.05017985,
         0.05053177, 0.05118456, 0.0514403 , 0.05136222, 0.0507592 ,
         0.05095532, 0.05082334, 0.05006258])})

In [27]:
frame_nums = train_results_ind["bbox"]["272"][IndividualDataFormat.frame_num]
frame_nums[1476]

1776

In [47]:
frame_nums = train_results_ind["bbox"]["272"][IndividualDataFormat.frame_num]
frame_nums[3653]

3953

# Test Data

In [43]:
peaks = find_peaks(test_results_ind["bbox"]["56"][IndividualDataFormat.loss_r], height=0.08)
peaks

(array([572, 575, 577, 624, 628, 642, 644, 647, 649, 653, 658, 668]),
 {'peak_heights': array([0.08133359, 0.08125252, 0.08139763, 0.08130525, 0.08106861,
         0.0884316 , 0.08869907, 0.08786689, 0.08842146, 0.09056663,
         0.09098689, 0.08607876])})

In [44]:
frame_nums = test_results_ind["bbox"]["56"][IndividualDataFormat.frame_num]
frame_nums[658]

1399

In [40]:
peaks = find_peaks(test_results_ind["local"]["56"][IndividualDataFormat.loss_r], height=0.08, prominence=0.02)
peaks

(array([296, 931]),
 {'peak_heights': array([0.20067677, 0.09856948]),
  'prominences': array([0.15550243, 0.03830859]),
  'left_bases': array([  0, 679]),
  'right_bases': array([ 679, 1215])})

In [39]:
np.array(frame_nums)[indices]

array([1037, 2010])